In [1]:
# --- This is to customise the notebook's appearance --- #
# Dependency: conda install -c conda-forge jupyterthemes
# More information: !jt --help
# See also: https://github.com/dunovank/jupyter-themes
try:
    !jt -N -t onedork -tf merriserif -fs 10 -tfs 12 -ofs 10 -mathfs 105 -altmd
except:
    pass

In [2]:
# --- This is not needed if NXP is installed --- #
import sys
import os.path as op

# add the src/ directory to the Python path
sys.path.insert(0,op.realpath('../src'))

# README example

This is the example of a simple LaTeX-like language provided in the README file of NXP.

In [3]:
with open('simple-latex.txt') as fh:
    print(fh.read())

Inspirational quote:
\quote{
    Time you enjoy wasting is \it{not} wasted time.
}

Command without a body \command, or with an empty one \command{}.


In [4]:
import nxp

# define these rules separately so they can be re-used
backslash = [ r'\\\\', ('rep','\\') ] 
command = [ r'\\(\w+)', ('open','command'), ('tag','cmd') ] 

# create a parser
parser = nxp.make_parser({
	'lang': {
		'main': [
			backslash,  # replace escaped backslashes
			command     # open "command" scope if we find something like '\word'
		],
		'command': { # the "command" scope
			'main': [
				[ r'\{', ('open','command.body'), ('tag','body') ],
					# open "body" subscope if command is followed by '{'
				[ None, 'close' ] 
					# otherwise close the scope
			],
			'body': [ # the "command.body" scope
				backslash,
				[ r'\\\{', ('rep','{') ],
				[ r'\\\}', ('rep','}') ],
					# deal with escapes before looking for a nested command
				command, 
					# look for nested commands
				[ r'\}', ('tag','/body'), ('close',2) ]
					# the command ends when the body ends: close both scopes
			]
		}
	}
})

print(nxp.parsefile( parser, 'simple-latex.txt' ))

+ Scope("main"): 3 element(s)
	[0] Scope("command"): 2 element(s)
		[0] \\(\w+)
			(0) (1, 0) - (1, 6) \quote
		[1] Scope("command.body"): 3 element(s)
			[0] \{
				(0) (1, 6) - (1, 7) {
			[1] Scope("command"): 2 element(s)
				[0] \\(\w+)
					(0) (2, 30) - (2, 33) \it
				[1] Scope("command.body"): 2 element(s)
					[0] \{
						(0) (2, 33) - (2, 34) {
					[1] \}
						(0) (2, 37) - (2, 38) }
			[2] \}
				(0) (3, 0) - (3, 1) }
	[1] Scope("command"): 1 element(s)
		[0] \\(\w+)
			(0) (5, 23) - (5, 31) \command
	[2] Scope("command"): 2 element(s)
		[0] \\(\w+)
			(0) (5, 54) - (5, 62) \command
		[1] Scope("command.body"): 2 element(s)
			[0] \{
				(0) (5, 62) - (5, 63) {
			[1] \}
				(0) (5, 63) - (5, 64) }
